LLMChain: 接受用户输入，使用PromptTemplate对其进行格式化，然后将格式化后的响应传递给LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
print(prompt.format(product="colorful socks"))

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
 
print(chain.run("colorful socks")) # run方法返回值为字符串

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

prompt=PromptTemplate(
    template="What is a good name for a company that makes {product}?",
    input_variables=["product"],
)

human_message_prompt = HumanMessagePromptTemplate(prompt=prompt)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

print(chain({"product":"colorful socks"})) # chain.run("colorful socks"), chain.run(product="colorful socks")

SimpleSequentialChain: 顺序链形式，每个步骤都有一个单一的输入/输出，一个步骤的输出是下一个步骤的输入

测试样例: 计算莱昂纳多·迪卡普里奥的女朋友的年龄的2次方是多少?

步骤: 
1. 查找莱昂纳多·迪卡普里奥的女朋友是谁
2. 查找查找莱昂纳多·迪卡普里奥的女朋友的年龄多大
3. 计算莱昂纳多·迪卡普里奥的女朋友的年龄的二次方

In [ ]:
from langchain.utilities import SerpAPIWrapper

def search_func(input, extra_string=''):
    print(input)
    if dict == type(input):
        input = ''.join(input.values())
    input = input + extra_string
    search = SerpAPIWrapper()
    info = search.run(input)
    print(input, ':', info)
    template = """
    已知信息: {info}
    请根据已知的信息回答: {input}
    注意: 回答的信息要包含在已知信息中，只输出答案，不要说明和解释信息
    """
    prompt_template = PromptTemplate(template=template, input_variables=["info", "input"])
    return {'output': llm(prompt=prompt_template.format(info=info, input=input))} # 让llm去生成精确的答案，对于非标准化的东西，程序很难精确获取想要的东西，这就是llm的优势(逻辑推导，脑力输出)

In [ ]:
name = search_func('莱昂纳多·迪卡普里奥的女朋友是谁？')
print(name)
age = search_func(f'{name}多大？')
print(age)
print(llm(f'{age}的2次方是多少？'))

In [ ]:
from langchain.chains import TransformChain, LLMChain
import functools

search_name_chain = TransformChain(input_variables=["input"], output_variables=["output"], transform=functools.partial(search_func, extra_string="的女朋友是谁？"))
search_age_chain = TransformChain(input_variables=["input"], output_variables=["output"], transform=functools.partial(search_func, extra_string="的年龄多大？"))
ll_math_chain = LLMChain(llm=llm, prompt=PromptTemplate(template="{age}的2次方是多少", input_variables=["age"]))

In [ ]:
from langchain.chains import SimpleSequentialChain

calculate_girlfriend_age = SimpleSequentialChain(chains=[search_name_chain, search_age_chain, ll_math_chain], verbose=True)
calculate_girlfriend_age.run(input="莱昂纳多·迪卡普里奥")

SequentialChain: 顺序链形式，允许多个输入/输出

测试样例: 将python code转化为C# code

步骤:
1. 第一个chain输入python，输出python code
2. 第二个chain输入C#和code，输出C# code

In [ ]:
language_template = PromptTemplate(
    input_variables = ['programming_language'], 
    template='Write main function and print hello word based on this programming language {programming_language}'
)

language_chain = LLMChain(llm=llm, prompt=language_template, verbose=True, output_key='code')

In [ ]:
translate_language_template = PromptTemplate(
    input_variables = ['translate_language', 'code'], 
    template='''Translate the following code for me to write in {translate_language} language:
    {code}
    '''
)

translate_chain = LLMChain(llm=llm, prompt=translate_language_template, verbose=True, output_key='translate_code')

使用Memory记录参数，并在多个chain之间传递

In [ ]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

sequential_chain = SequentialChain(
    memory=SimpleMemory(memories={"translate_language": "C#"}),
    chains = [language_chain, translate_chain], 
    input_variables=['programming_language'], 
    output_variables=['translate_code'],
    verbose=True)

sequential_chain.run(programming_language='python')
